In [1]:
import numpy as np

In [125]:
def unite_neighbors(D, weight):
    min_dist = np.inf
    for v1 in D:
        for v2 in D[v1]:
            if D[v1][v2] < min_dist:
                x, y = v1, v2
                min_dist = D[x][y]
    x, y = min(x, y), max(x, y)
    new_v = x + y
    x_neighbors = D.pop(x) if x in D else {}
    y_neighbors = D.pop(y) if y in D else {}
    
    to_remove = []
    for v1 in D:
        for v2 in D[v1]:
            if v2 == x:
                x_neighbors[v1] = D[v1][v2]
                to_remove.append((v1, v2))
            elif v2 == y:
                y_neighbors[v1] = D[v1][v2]
                to_remove.append((v1, v2))
                
    for v1, v2 in to_remove:
        D[v1].pop(v2)

    curr_keys = list(D.keys())
    D[new_v] = {}
    
    for v in curr_keys:
        if weight:
            D[new_v][v] = (x_neighbors[v] + y_neighbors[v]) / 2
        else:
            D[new_v][v] = (x_neighbors[v] * len(x) + y_neighbors[v] * len(y)) / (len(x) + len(y))
    
    return x, y, min_dist / 2

In [126]:
def pgma(D, weight):
    nwck = {}
    offset = {}
    while len(D) > 1:
        x, y, dist = unite_neighbors(D, weight)
        if x not in offset:
            offset[x] = 0
        if y not in offset:
            offset[y] = 0
        nwck[x+y] = '({x}:{dx}, {y}:{dy})'.format(x=nwck[x] if x in nwck else x, y=nwck[y] if y in nwck else y,
                                                  dx=dist - offset[x], dy=dist - offset[y])
        offset[x+y] = dist
        
    return nwck[list(D.keys())[0]]

In [122]:
D = {
    'A': {'B': 5, 'C': 4, 'D': 7, 'E': 6, 'F': 8},
    'B': {'C': 7, 'D': 10, 'E': 9, 'F': 11},
    'C': {'D': 7, 'E': 6, 'F': 8},
    'D': {'E': 5, 'F': 9},
    'E': {'F': 8}
}

In [129]:
D = {
    'K': {'L': 16, 'M': 16, 'N': 10},
    'L': {'M': 8, 'N': 8},
    'M': {'N': 4}
}

In [130]:
pgma(D, False)

'(K:7.0, (L:4.0, (M:2.0, N:2.0):2.0):3.0)'

In [173]:
def get_vert(D):
    vert = set([])
    for v1 in D:
        vert.add(v1)
        for v2 in D[v1]:
            vert.add(v2)
    return list(vert)

def M(D, a, b):
    dist = 0
    if a in D:
        for v in D[a]:
            if v != b:
                dist += D[a][v]
    for v in D:
        if v != b and a in D[v]:
            dist += D[v][a]
        
    return dist / (len(get_vert(D)) - 2)

def join_neighbors(D):
    vert = get_vert(D)
    min_dist = np.inf
    for i, v1 in enumerate(vert[:-1]):
        for v2 in vert[i+1:]:
            dist = (D[v1][v2] if v1 in D and v2 in D[v1] else D[v2][v1]) - M(D, v1, v2) - M(D, v2, v1)
            if dist < min_dist:
                x, y = v1, v2
                min_dist = dist
                
    x, y = min(x, y), max(x, y)
    dist = D[x][y] if x in D and y in D[x] else D[y][x]
    x_dist = (dist + M(D, x, y) - M(D, y, x)) / 2
    y_dist = (dist + M(D, y, x) - M(D, x, y)) / 2
    
    new_v = x + y
    x_neighbors = D.pop(x) if x in D else {}
    y_neighbors = D.pop(y) if y in D else {}
    
    to_remove = []
    for v1 in D:
        for v2 in D[v1]:
            if v2 == x:
                x_neighbors[v1] = D[v1][v2]
                to_remove.append((v1, v2))
            elif v2 == y:
                y_neighbors[v1] = D[v1][v2]
                to_remove.append((v1, v2))
                
    for v1, v2 in to_remove:
        D[v1].pop(v2)

    curr_keys = list(D.keys())
    D[new_v] = {}
    
    for v in curr_keys:
        D[new_v][v] = (x_neighbors[v] + y_neighbors[v] - dist) / 2
    
    print(D)
    
    return x, y, x_dist, y_dist

In [177]:
def nj(D):
    nwck = {}
    offset = {}
    while len(get_vert(D)) > 2:
        x, y, x_dist, y_dist = join_neighbors(D)
        print('new cluster: {cl}, D({cl}, {x})={xd}, D({cl}, {y})={yd}\n'.format(cl=x+y, x=x, y=y, xd=x_dist,
                                                                               yd=y_dist))

In [178]:
D = {
    'K': {'L': 16, 'M': 16, 'N': 10},
    'L': {'M': 8, 'N': 8},
    'M': {'N': 4}
}

In [179]:
nj(D)

{'K': {'L': 16}, 'L': {}, 'MN': {'K': 11.0, 'L': 6.0}}
new cluster: MN, D(MN, M)=3.5, D(MN, N)=0.5

{'K': {}, 'LMN': {'K': 10.5}}
new cluster: LMN, D(LMN, L)=5.5, D(LMN, MN)=0.5

